# [실습5] 금속분말 생성공정 최적화를 위한 딥러닝 심화


---

## 실습 목표
---
- 모델의 일반화에 대해 이해합니다.
- 일반화를 위한 방법들을 배워봅니다.
- 조기 종료 방법을 배워봅니다.
- 가중치 규제 방법을 배워봅니다.
- 앙상블 모델을 배워봅니다.
- 정규화를 배워봅니다.
- 데이터 증강 기법을 배워봅니다.

## 실습 목차
---
1. **일반화를 위한 방법들:** 일반화를 위한 여러 방법들을 사용해보고 성능을 비교합니다.

2. **조기 종료:** 조기 종료 방법을 수행해보고, 성능을 비교해봅니다,

3. **가중치 규제:** 가중치 규제 방법을 수행해보고, 성능을 비교해봅니다.

4. **앙상블 모델:** 앙상블 모델을 구현해봅니다.

5. **Dropout:** Dropout을 수행해보고, 성능을 비교해봅니다.

6. **정규화:** 정규화를 수행해보고, 성능을 비교해봅니다.

7. **데이터 증강 기법:** 데이터 증강기법을 수행해보고, 성능을 비교해봅니다.

## 실습 개요
---

이번 실습에서는 다양한 일반화를 위한 방법들을 수행해봅니다.

## 1. 일반화를 위한 방법들
---
금속분말 데이터셋을 이용하여 일반화 방법들을 수행해보겠습니다.


### 1.1 라이브러리 불러오기

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
import json
import sklearn.metrics

### 1.2 데이터셋 읽어오기

In [ ]:
stage1 = {
    'train_X': np.load('./Data/train_data_stage1_X.npy'),
    'train_y': np.load('./Data/train_data_stage1_y.npy'),
    'valid_X': np.load('./Data/valid_data_stage1_X.npy'),
    'valid_y': np.load('./Data/valid_data_stage1_y.npy'),
    'test_X': np.load('./Data/test_data_stage1_X.npy'),
    'test_y': np.load('./Data/test_data_stage1_y.npy'),
}

stage2 = {
    'train_X': np.load('./Data/train_data_stage2_X.npy'),
    'train_y': np.load('./Data/train_data_stage2_y.npy'),
    'valid_X': np.load('./Data/valid_data_stage2_X.npy'),
    'valid_y': np.load('./Data/valid_data_stage2_y.npy'),
    'test_X': np.load('./Data/test_data_stage2_X.npy'),
    'test_y': np.load('./Data/test_data_stage2_y.npy'),
}

columns = json.load(open('./Data/valid_columns.json', 'r'))

### 1.3 데이터 표준화하기

### 1.3.1 Stage1 데이터 표준화하기

In [ ]:
stage1_X_mean = stage1['train_X'].mean(axis = 0)
stage1_y_mean = stage1['train_y'].mean(axis = 0)
print('입력값 평균:', stage1_X_mean)
print('출력값 평균:', stage1_y_mean)

In [ ]:
stage1_X_std = stage1['train_X'].std(axis = 0)
stage1_y_std = stage1['train_y'].std(axis = 0)
print('입력값 표준편차:', stage1_X_std)
print('출력값 표준편차:', stage1_y_std)

In [ ]:
# 학습 데이터 표준화
stage1['train_X'] = (stage1['train_X'] - stage1_X_mean) / stage1_X_std
stage1['train_y'] = (stage1['train_y'] - stage1_y_mean) / stage1_y_std
# 검증용 데이터 표준화
stage1['valid_X'] = (stage1['valid_X'] - stage1_X_mean) / stage1_X_std
stage1['valid_y'] = (stage1['valid_y'] - stage1_y_mean) / stage1_y_std
# 테스트 데이터 표준화
stage1['test_X'] = (stage1['test_X'] - stage1_X_mean) / stage1_X_std
stage1['test_y'] = (stage1['test_y'] - stage1_y_mean) / stage1_y_std

### 1.3.2 Stage2 데이터 표준화하기

In [ ]:
stage2_X_mean = stage2['train_X'].mean(axis = 0)
stage2_y_mean = stage2['train_y'].mean(axis = 0)
print('입력값 평균:', stage2_X_mean)
print('출력값 평균:', stage2_y_mean)

In [ ]:
stage2_X_std = stage2['train_X'].std(axis = 0)
stage2_y_std = stage2['train_y'].std(axis = 0)
print('입력값 표준편차:', stage2_X_std)
print('출력값 표준편차:', stage2_y_std)

In [ ]:
# 학습 데이터 표준화
stage2['train_X'] = (stage2['train_X'] - stage2_X_mean) / stage2_X_std
stage2['train_y'] = (stage2['train_y'] - stage2_y_mean) / stage2_y_std

# 검증용 데이터 표준화
stage2['valid_X'] = (stage2['valid_X'] - stage2_X_mean) / stage2_X_std
stage2['valid_y'] = (stage2['valid_y'] - stage2_y_mean) / stage2_y_std

# 테스트 데이터 표준화
stage2['test_X'] = (stage2['test_X'] - stage2_X_mean) / stage2_X_std
stage2['test_y'] = (stage2['test_y'] - stage2_y_mean) / stage2_y_std

### 1.4 학습 데이터 수에 따른 모델 성능 비교

In [ ]:
data_use = [0.01, 0.1, 0.5, 1]

학습데이터의 1%, 10%, 50%, 100% 를 사용한 모델의 성능을 비교해보겠습니다.

In [ ]:
for ratio in data_use:
    np.random.seed(0)
    random.seed(0)
    tf.random.set_seed(0)
    # 모델 정의
    MLP_model = tf.keras.Sequential([
        Input(shape = stage1['train_X'].shape[1]),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(stage1['train_y'].shape[1])
    ])
    
    # 모델 컴파일
    MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
    )
    
    # 학습 데이터 개수
    total_data = len(stage1['train_X'])
    len_data = int(total_data * ratio)
    # 모델 학습
    history = MLP_model.fit(stage1['train_X'][:len_data], stage1['train_y'][:len_data], 
                            epochs = 50 * int(1 / ratio),
                            batch_size = 16, verbose = 0)
    pred = MLP_model.predict(stage1['test_X'])
    r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
    print("R2 score (학습데이터 수: %d): %f"%(len_data, r2))

학습데이터가 많을수록 모델의 성능이 좋은 것을 확인하였습니다.

## 2. Early stop
검증용 데이터를 이용하여 모델이 과적합 되기 전에 학습을 중지해보겠습니다.

### 2.1 모델 정의

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# early stop 을 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# early stop 을 사용할 모델입니다.
MLP_model_es = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 2.2 모델 학습 방법 설정

In [ ]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_es.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

### 2.3 모델 학습

먼저, early stop 을 사용하지 않을 모델부터 학습합니다. 실습시간을 고려하여 전체 학습데이터 중 1000개만 사용하겠습니다.

In [ ]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                        validation_data = (stage1['valid_X'], stage1['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

다음으로, early stop을 사용할 모델을 학습합니다.

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50)
history_es = MLP_model_es.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], 
                           validation_data = (stage1['valid_X'], stage1['valid_y']),
                           epochs = 500, batch_size = 16, verbose = 2,
                          callbacks = [es])

### 2.4 모델 예측 및 평가

In [ ]:
pred = MLP_model.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_es.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
plt.plot(history.history['loss'], label = 'train w/o EarlyStop')
plt.plot(history_es.history['loss'], label = 'train w/ EarlyStop')
plt.plot(history.history['val_loss'], label = 'valid w/o EarlyStop')
plt.plot(history_es.history['val_loss'], label = 'valid w/ EarlyStop')
plt.legend()
plt.show()

검증용 데이터셋을 이용하여 early stop 하였을 때 모델의 성능이 더 좋습니다.

### [TODO] Stage2에 대해 Early stop 을 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# early stop 을 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

# early stop 을 사용할 모델입니다.
MLP_model_es = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

In [ ]:
# MLP 모델을 컴파일 합니다.
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_es.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

In [ ]:
# Early stop 을 적용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                        epochs = 500, 
                        batch_size = 16, 
                        verbose = 2)

In [ ]:
# 다음으로, Early stop 을 적용할 모델을 학습합니다.
es = _____________________
history_es = MLP_model_es.fit(________________, _________________, 
                           validation_data = (____________, ______________),
                           epochs = 500, batch_size = 16, verbose = 2,
                          callbacks = ____)

In [ ]:
# Early stop 을 사용하지 않은 모델을 평가합니다.
pred = MLP_model.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
# Early stop 을 사용한 모델을 평가합니다.
pred = ___________________________
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
plt.plot(history.history['loss'], label = 'train w/o EarlyStop')
plt.plot(history_es.history['loss'], label = 'train w/ EarlyStop')
plt.plot(history.history['val_loss'], label = 'valid w/o EarlyStop')
plt.plot(history_es.history['val_loss'], label = 'valid w/ EarlyStop')
plt.legend()
plt.show()

Stage2 의 데이터에 대해서는 early stop을 사용하였을 때, r2-score 가 더 낮은 것을 확인할 수 있습니다. 모든 데이터셋에 대해 특정 방법이 항상 우세하지는 않습니다. 데이터마다 어떠한 방법을 선택하고, 어떤 하이퍼 파라미터를 선택할지 신중하게 선택할 필요가 있습니다.

## 3. 가중치 규제
---
금속분말 데이터셋을 이용하여 가중치 규제를 모델에 적용해보겠습니다.


### 3.1 모델 정의

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# 가중치 규제를 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 가중치규제를 사용할 모델입니다. L1 regularizer을 적용해봅니다.
MLP_model_reg = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)), 
    tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 3.2 모델 학습 방법 설정

In [ ]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_reg.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

### 3.3 모델 학습

In [ ]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                        validation_data = (stage1['valid_X'], stage1['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
history_reg = MLP_model_reg.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                            validation_data = (stage1['valid_X'], stage1['valid_y']),
                            epochs = 500,
                            batch_size = 16,
                            verbose = 2)

In [ ]:
pred = MLP_model.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_reg.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

가중치 규제를 사용한 모델에서 성능이 더 좋았습니다.

### [TODO] Stage2에 대해 가중치규제를 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# 가중치 규제를 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

# 가중치규제를 사용할 모델입니다. L1 regularizer을 적용해봅니다.
MLP_model_reg = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu', ___________________________________, 
    tf.keras.layers.Dense(64, activation = 'relu', ___________________________________,
    tf.keras.layers.Dense(32, activation = 'relu', ___________________________________,
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

In [ ]:
# MLP 모델을 컴파일 합니다.
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_reg.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

In [ ]:
# 가중치규제를 사용하지 않는 모델을 학습합니다.
history = MLP_model.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# 다음으로, 가중치규제를 사용하는 모델을 학습합니다.
history_reg = MLP_model_reg.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                            validation_data = (stage2['valid_X'], stage2['valid_y']),
                            epochs = 500,
                            batch_size = 16,
                            verbose = 2)

In [ ]:
# 가중치 규제를 사용하지 않은 모델을 평가합니다.
pred = MLP_model.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_reg.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

## 4. 앙상블 모델
---
금속분말 데이터셋을 이용하여 앙상블 모델을 학습해보겠습니다.


### 4.1 Weak 모델 정의

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
n_estimators = 10
models = []
for i in range(n_estimators):
    model = tf.keras.Sequential([
        Input(shape = stage1['train_X'].shape[1]),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(stage1['train_y'].shape[1])
    ])
    models.append(model)

### 4.2 모델 컴파일

In [ ]:
for model in models:
    model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)

### 4.3 모델 학습

In [ ]:
len_training_data = len(stage1['train_X'])
len_subset = int(len_training_data * 0.25)
histories =[]
idxs = np.arange(len_training_data)
for model in models:
    train_X = stage1['train_X'][idxs][:len_subset]
    train_y = stage1['train_y'][idxs][:len_subset]
    history = model.fit(train_X, train_y, epochs = 50, batch_size = 16, verbose = 2)
    histories.append(history)
    np.random.shuffle(idxs)

### 4.4 모델 예측 및 평가

In [ ]:
preds = 0
for i, model in enumerate(models):
    pred = model.predict(stage1['test_X'])
    r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
    print("%d 번째 weak model - R2 score: %f"%(i+1, r2))
    preds += pred
preds /= len(models)
r2 = sklearn.metrics.r2_score(stage1['test_y'], preds)
print("앙상블 모델 R2 score: %f"%r2)

앙상블 모델이 각각의 약한 모델보다 성능이 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 앙상블을 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# Weak 모델 정의
n_estimators = 10
models = []
# Stage1 에 사용했던 앙상블 모델을 그대로 사용해봅시다.
for i in range(n_estimators):
    model = tf.keras.Sequential([
        Input(shape = stage2['train_X'].shape[1]),
        ___________________________________,
        ___________________________________,
        tf.keras.layers.Dense(stage2['train_y'].shape[1])
    ])
    models.append(model)

In [ ]:
# 모델 컴파일
for model in models:
    model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)

In [ ]:
# 모델 학습
len_training_data = len(stage2['train_X'])
len_subset = int(len_training_data * 0.25)
histories =[]
idxs = np.arange(len_training_data)
for model in models:
    train_X = _________________________
    train_y = _________________________
    history = model.fit(train_X, train_y, epochs = 50, batch_size = 16, verbose = 2)
    histories.append(history)
    np.random.shuffle(idxs)

In [ ]:
# 모델 예측 및 평가
preds = 0
for i, model in enumerate(models):
    pred = model.predict(stage2['test_X'])
    r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
    print("%d 번째 weak model - R2 score: %f"%(i+1, r2))
    preds += pred
preds /= len(models)
r2 = sklearn.metrics.r2_score(stage2['test_y'], preds)
print("앙상블 모델 R2 score: %f"%r2)

## 5. Dropout
---
금속분말 데이터셋을 이용하여 Dropout 을 적용해보겠습니다.


### 5.1 모델 정의

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# Dropout을 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# Dropout을 사용할 모델입니다. rate는 0.2로 설정합니다.
MLP_model_dropout = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dropout(rate=0.2), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 5.2 모델 컴파일

In [ ]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_dropout.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

### 5.3 모델 학습

In [ ]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                        validation_data = (stage1['valid_X'], stage1['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
history_dropout = MLP_model_dropout.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                                validation_data = (stage1['valid_X'], stage1['valid_y']),
                                epochs = 500,
                                batch_size = 16,
                                verbose = 2)

### 5.4 모델 예측 및 평가

In [ ]:
pred = MLP_model.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_dropout.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

Dropout 을 사용한 모델이 더 성능이 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 Dropout을 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# Dropout을 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

# Dropout을 사용할 모델입니다. rate는 0.2로 설정합니다.
MLP_model_dropout = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    ___________________________________, 
    tf.keras.layers.Dense(64, activation = 'relu'),
    ___________________________________,
    tf.keras.layers.Dense(32, activation = 'relu'),
    ___________________________________,
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

In [ ]:
# 모델 컴파일
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_dropout.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

In [ ]:
# Dropout 을 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# Dropout 을 사용할 모델을 학습합니다.
history_dropout = MLP_model_dropout.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                                        epochs = 500,
                                        batch_size = 16,
                                        verbose = 2)

In [ ]:
# Dropout 을 사용하지 않은 모델을 평가합니다.
pred = MLP_model.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
# Dropout 을 사용한 않은 모델을 평가합니다.
pred = MLP_model_dropout.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

## 6. 정규화
---
금속분말 데이터셋을 이용하여 정규화를 수행해보겠습니다.

### 6.1 모델 정의

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# 정규화를 하지 않을 모델입니다. Dropout에서 학습한 모델과 같은 구조를 사용하겠습니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 정규화를 사용할 모델입니다. Batch normalization 을 사용해보겠습니다.
MLP_model_ln = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 6.2 모델 컴파일

In [ ]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)
MLP_model_ln.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)

### 6.3 모델 학습

In [ ]:
# 정규화를 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                        validation_data = (stage1['valid_X'], stage1['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# 정규화를 사용할 모델을 학습합니다.
history_ln = MLP_model_ln.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                           validation_data = (stage1['valid_X'], stage1['valid_y']),
                           epochs = 500,
                           batch_size = 16,
                           verbose = 2)

### 6.4 모델 예측 및 평가

In [ ]:
pred = MLP_model.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_ln.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

정규화를 사용한 모델에서 성능이 더 좋습니다.

### [TODO] Stage2에 대해 정규화를 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
#MLP 모델을 설정합니다.
# 정규화를 하지 않을 모델입니다. 
MLP_model = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

# 정규화를 사용할 모델입니다. Batch normalization 을 사용해보겠습니다.
MLP_model_ln = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128), 
    ___________________________________, 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    ___________________________________, 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    ___________________________________, 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

In [ ]:
# 모델 컴파일
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)
MLP_model_ln.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
)

In [ ]:
# 정규화를 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# 정규화를 사용할 모델을 학습합니다.
history = MLP_model_ln.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                           validation_data = (stage2['valid_X'], stage2['valid_y']),
                           epochs = 500,
                           batch_size = 16,
                           verbose = 2)

In [ ]:
# 정규화를 사용하지 않은 모델을 평가합니다.
pred = MLP_model.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
# 정규화를 사용한 모델을 평가합니다.
pred = MLP_model_ln.predict(stage2['test_X'])
r2 = sklearn.metrics.r2_score(stage2['test_y'], pred)
print("R2 score: %f"%r2)

## 7. 데이터 증강 기법
---
금속분말 데이터셋을 이용하여 정규화를 수행해보겠습니다.


### 7.1 모델 정의

In [ ]:
# MLP 모델을 설정합니다.
# 데이터 증강을 하지 않을 모델입니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 데이터 증강 기법을 사용할 모델입니다. 데이터에 임의의 가우시안 노이즈를 추가해보겠습니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_aug = tf.keras.Sequential([
    Input(shape = stage1['train_X'].shape[1]),
    tf.keras.layers.GaussianNoise(stddev = 0.1),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 7.2 모델 컴파일

In [ ]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_aug.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

### 7.3 모델 학습

In [ ]:
# 데이터 증강기법을 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                        validation_data = (stage1['valid_X'], stage1['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# 데이터 증강기법을 사용할 모델을 학습합니다.
history = MLP_model_aug.fit(stage1['train_X'][:1000], stage1['train_y'][:1000],
                            validation_data = (stage1['valid_X'], stage1['valid_y']),
                            epochs = 500,
                            batch_size = 16,
                            verbose = 2)

### 7.4 모델 예측 및 평가

In [ ]:
pred = MLP_model.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

In [ ]:
pred = MLP_model_aug.predict(stage1['test_X'])

In [ ]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

데이터 증강 기법을 사용한 모델이 성능이 더 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 데이터 증강기법을 적용해봅니다.

In [ ]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
# MLP 모델을 설정합니다.
# 데이터 증강을 하지 않을 모델입니다.
MLP_model = tf.keras.Sequential([
    Input(shape = stage2['train_X'].shape[1]),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage2['train_y'].shape[1])
])

# 데이터 증강 기법을 사용할 모델입니다. 데이터에 Stage1에서 추가했던 가우시안 노이즈를 추가해보겠습니다.
MLP_model_aug = ______________________

In [ ]:
# 모델 컴파일
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)
MLP_model_aug.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
)

In [ ]:
# 데이터 증강기법을 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                        validation_data = (stage2['valid_X'], stage2['valid_y']),
                        epochs = 500,
                        batch_size = 16,
                        verbose = 2)

In [ ]:
# 데이터 증강기법을 사용할 모델을 학습합니다.
history_aug = MLP_model_aug.fit(stage2['train_X'][:1000], stage2['train_y'][:1000],
                            validation_data = (stage2['valid_X'], stage2['valid_y']),
                            epochs = 500,
                            batch_size = 16,
                            verbose = 2)

In [ ]:
# 정규화를 사용하지 않은 모델을 평가합니다.
pred = ________________________________
r2 = ________________________________
print("R2 score: %f"%r2)

In [ ]:
# 정규화를 사용한 모델을 평가합니다.
pred = ________________________________
r2 = ________________________________
print("R2 score: %f"%r2)

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>